In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
pd.__version__

'0.23.4'

In [3]:
df_off = pd.read_csv('data/ccf_offline_stage1_train.csv')
df_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')

In [4]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN


In [5]:
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706


In [6]:
coupon_use_count = df_off[df_off.Coupon_id.notnull() & df_off.Date.notnull()].shape[0]
coupon_notuse_count = df_off[df_off.Coupon_id.notnull() & df_off.Date.isnull()].shape[0]
nocoupon_buy_count = df_off[df_off.Coupon_id.isnull() & df_off.Date.notnull()].shape[0]
nocoupon_nobuy_count = df_off[df_off.Coupon_id.isnull() & df_off.Date.isnull()].shape[0]

print('获得优惠卷，并使用：', coupon_use_count)
print('获得优惠卷，没使用：', coupon_notuse_count)
print('没获得优惠卷，消费：', nocoupon_buy_count)
print('没获得优惠卷，没消费：', nocoupon_nobuy_count)

获得优惠卷，并使用： 75382
获得优惠卷，没使用： 977900
没获得优惠卷，消费： 701602
没获得优惠卷，没消费： 0


### 清理掉没获得优惠劵的条目

In [7]:
df_off = df_off[df_off['Coupon_id'].notnull()]

### 用到的特征

#### 用户特征
- 用户领取优惠券次数
- 用户获得优惠券但没有消费的次数
- 用户获得优惠券并核销次数
- 用户领取优惠券后进行核销率

#### 商家特征
- 商家优惠券被领取次数
- 商家优惠券被领取后不核销次数
- 商家优惠券被领取后核销次数
- 商家优惠券被领取后核销率

#### 优惠卷特征
- 优惠券类型(直接优惠为0, 满减为1)
- 优惠券折率
- 历史出现次数
- 历史核销次数
- 历史核销率

#### 用户与商家交互
- 用户领取商家的优惠券次数
- 用户领取商家的优惠券后不核销次数
- 用户领取商家的优惠券后核销次数
- 用户领取商家的优惠券后核销率

#### 环境特征
- 月份
- 星期几
- 是否是周末

### 用户特征

In [8]:
df_user = pd.DataFrame()
df_user['User_id'] = df_off.User_id.unique()
df_user.sort_values(by='User_id', ascending=True, inplace=True)
df_user.index = range(0, df_user.shape[0])
df_user.head(3)

,User_id
0,4
1,35
2,36


#### 用户领取优惠券次数 Coupon_counts

In [9]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN


In [10]:
df_coupon_counts = df_off.groupby('User_id', as_index=False).Coupon_id.count()
df_coupon_counts.columns = ['User_id', 'Coupon_counts']
df_coupon_counts.head(3)

,User_id,Coupon_counts
0,4,2
1,35,4
2,36,2


In [11]:
df_user = pd.merge(df_user, df_coupon_counts, how='outer', on='User_id')
df_user.head(2)

,User_id,Coupon_counts
0,4,2
1,35,4


#### 用户获得优惠券但没有消费的次数 Coupon_nouse_counts

In [12]:
df_coupon_nouse_counts = df_off[df_off.Coupon_id.notnull() & df_off.Date.isnull()].groupby('User_id', as_index=False).Coupon_id.count()
df_coupon_nouse_counts.columns = ['User_id', 'Coupon_nouse_counts']
df_user = pd.merge(df_user, df_coupon_nouse_counts, how='outer', on='User_id')
df_user.head(2)

,User_id,Coupon_counts,Coupon_nouse_counts
0,4,2,2.0
1,35,4,4.0


#### 用户获得优惠券并核销次数 Coupon_use_counts

In [13]:
df_coupon_use_counts = df_off[df_off.Coupon_id.notnull() & df_off.Date.notnull()].groupby('User_id', as_index=False).Coupon_id.count()
df_coupon_use_counts.columns = ['User_id', 'Coupon_use_counts']
df_user = pd.merge(df_user, df_coupon_use_counts, how='outer', on='User_id')
df_user.head(2)

,User_id,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts
0,4,2,2.0,NaN
1,35,4,4.0,NaN


#### 用户领取优惠券后进行核销率 Coupon_use_rate

In [14]:
df_user.fillna(0, inplace=True)
df_user.head(3)

,User_id,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts
0,4,2,2.0,0.0
1,35,4,4.0,0.0
2,36,2,2.0,0.0


In [15]:
df_user['Coupon_use_rate'] = df_user.Coupon_use_counts / df_user.Coupon_counts
df_user.Coupon_use_rate.fillna(0, inplace=True)
df_user.head(3)

,User_id,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate
0,4,2,2.0,0.0,0.0
1,35,4,4.0,0.0,0.0
2,36,2,2.0,0.0,0.0


### 商户特征

In [16]:
df_merchant = pd.DataFrame()
df_merchant['Merchant_id'] = df_off.Merchant_id.unique()
df_merchant.sort_values(by='Merchant_id', ascending=True, inplace=True)
df_merchant.index = range(0, df_merchant.shape[0])
df_merchant.head(3)

,Merchant_id
0,2
1,3
2,4


#### 商家优惠券被领取次数 Merchant_coupon_counts

In [17]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN


In [18]:
df_merchant_coupon_counts = df_off.groupby('Merchant_id', as_index=False).Date_received.count()
df_merchant_coupon_counts.columns = ['Merchant_id', 'Merchant_coupon_counts']
df_merchant = pd.merge(df_merchant, df_merchant_coupon_counts, how='outer')
df_merchant.head(3)

,Merchant_id,Merchant_coupon_counts
0,2,7
1,3,10
2,4,7


#### 商家优惠券被领取后不核销次数 Merchant_coupon_nouse_counts

In [19]:
df_merchant_coupon_nouse_counts = df_off[df_off['Date_received'].notnull() & df_off['Date'].isnull()].groupby('Merchant_id', as_index=False).Date_received.count()
df_merchant_coupon_nouse_counts.columns = ['Merchant_id', 'Merchant_coupon_nouse_counts']
df_merchant = pd.merge(df_merchant, df_merchant_coupon_nouse_counts, how='outer')
df_merchant.Merchant_coupon_nouse_counts.fillna(0, inplace=True)
df_merchant.head(3)

,Merchant_id,Merchant_coupon_counts,Merchant_coupon_nouse_counts
0,2,7,7.0
1,3,10,9.0
2,4,7,2.0


#### 商家优惠券被领取后核销次数 Merchant_coupon_use_counts

In [20]:
df_merchant_coupon_use_counts = df_off[df_off['Date_received'].notnull() & df_off['Date'].notnull()].groupby('Merchant_id', as_index=False).Date_received.count()
df_merchant_coupon_use_counts.columns = ['Merchant_id', 'Merchant_coupon_use_counts']
df_merchant = pd.merge(df_merchant, df_merchant_coupon_use_counts, how='outer')
df_merchant.Merchant_coupon_use_counts.fillna(0, inplace=True)
df_merchant.head(3)

,Merchant_id,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts
0,2,7,7.0,0.0
1,3,10,9.0,1.0
2,4,7,2.0,5.0


#### 商家优惠券被领取后核销率 Merchant_coupon_use_rate

In [21]:
df_merchant['Merchant_coupon_use_rate'] = df_merchant.Merchant_coupon_use_counts / df_merchant.Merchant_coupon_counts
df_merchant.Merchant_coupon_use_rate.fillna(0, inplace=True)
df_merchant.head(3)

,Merchant_id,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate
0,2,7,7.0,0.0,0.000000
1,3,10,9.0,1.0,0.100000
2,4,7,2.0,5.0,0.714286


### 优惠劵特征

#### 优惠券类型(直接优惠为0, 满减为1) Coupon_type

In [22]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN


In [23]:
df_off.Discount_rate.unique()

array(['150:20', '20:1', '200:20', '30:5', '50:10', '10:5', '100:10',
       '200:30', '20:5', '30:10', '50:5', '150:10', '100:30', '200:50',
       '100:50', '300:30', '50:20', '0.9', '10:1', '30:1', '0.95', '100:5',
       '5:1', '100:20', '0.8', '50:1', '200:10', '300:20', '100:1',
       '150:30', '300:50', '20:10', '0.85', '0.6', '150:50', '0.75', '0.5',
       '200:5', '0.7', '30:20', '300:10', '0.2', '50:30', '200:100',
       '150:5'], dtype=object)

In [24]:
def parse_discount_rate_type(item):
    item = str(item)
    if item == 'nan':
        return np.nan
    elif ':' in item:
        return 1
    else:
        return 0

In [25]:
df_off['Coupon_type'] = df_off.Discount_rate.apply(parse_discount_rate_type)
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1


In [26]:
df_test['Coupon_type'] = df_test.Discount_rate.apply(parse_discount_rate_type)
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Coupon_type
0,4129537,450,9983,30:5,1.0,20160712,1
1,6949378,1300,3429,30:5,NaN,20160706,1


#### 优惠券折率 Coupon_rate

In [27]:
def parse_discount_rate(item):
    item = str(item)
    
    if item == 'nan':
        return np.nan
    elif ':' in item:
        left, right = item.split(':')
        return (float(left) - float(right)) / float(left)
    else:
        return float(item)

In [28]:
df_off['Coupon_rate'] = df_off.Discount_rate.apply(parse_discount_rate)
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000


In [29]:
df_test['Coupon_rate'] = df_test.Discount_rate.apply(parse_discount_rate)
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Coupon_type,Coupon_rate
0,4129537,450,9983,30:5,1.0,20160712,1,0.833333
1,6949378,1300,3429,30:5,NaN,20160706,1,0.833333


#### 历史出现次数 Coupon_history_counts

In [30]:
df_coupon_history_counts = df_off.groupby('Coupon_id', as_index=False).Date_received.count()
df_coupon_history_counts.columns = ['Coupon_id', 'Coupon_history_counts']
df_coupon = df_coupon_history_counts.copy()
df_coupon.head(2)

,Coupon_id,Coupon_history_counts
0,1.0,5
1,2.0,2


#### 历史核销次数 Coupon_hisotry_use_counts

In [31]:
df_coupon_history_use = df_off.groupby('Coupon_id', as_index=False).Date.count()
df_coupon_history_use.columns = ['Coupon_id', 'Coupon_hisotry_use_counts']
df_coupon = pd.merge(df_coupon, df_coupon_history_use, on='Coupon_id', how='outer')
df_coupon.head(2)

,Coupon_id,Coupon_history_counts,Coupon_hisotry_use_counts
0,1.0,5,1
1,2.0,2,1


#### 历史核销率 Coupon_hisotry_rate

In [32]:
df_coupon['Coupon_history_rate'] = df_coupon.Coupon_hisotry_use_counts / df_coupon.Coupon_history_counts
df_coupon.head(2)

,Coupon_id,Coupon_history_counts,Coupon_hisotry_use_counts,Coupon_history_rate
0,1.0,5,1,0.2
1,2.0,2,1,0.5


### 用户与商家交互

In [33]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000


#### 用户领取商家的优惠券次数 User_merchant_coupon_counts

In [34]:
df_user_merchant = df_off.groupby(['User_id', 'Merchant_id'], as_index=False).Date_received.count().copy()
df_user_merchant.rename(columns={'Date_received': 'User_merchant_coupon_counts'}, inplace=True)
df_user_merchant.head(3)

,User_id,Merchant_id,User_merchant_coupon_counts
0,4,1433,1
1,4,1469,1
2,35,3381,4


#### 用户领取商家的优惠券后不核销次数 User_merchant_counon_nouse

In [35]:
df_user_merchant_coupon_nouse = df_off[df_off['Date_received'].notnull() & df_off['Date'].isnull()].groupby(['User_id', 'Merchant_id'], as_index=False).Date_received.count()
df_user_merchant_coupon_nouse.columns = ['User_id', 'Merchant_id', 'User_merchant_coupon_nouse']
df_user_merchant = pd.merge(df_user_merchant, df_user_merchant_coupon_nouse, on=['User_id', 'Merchant_id'], how='outer')
df_user_merchant.User_merchant_coupon_nouse.fillna(0, inplace=True)
df_user_merchant.User_merchant_coupon_nouse = df_user_merchant.User_merchant_coupon_nouse.astype(int)
df_user_merchant.head(2)

,User_id,Merchant_id,User_merchant_coupon_counts,User_merchant_coupon_nouse
0,4,1433,1,1
1,4,1469,1,1


#### 用户领取商家的优惠券后核销次数  User_merchant_coupon_use

In [36]:
df_user_merchant_coupon_use = df_off[df_off['Date_received'].notnull() & df_off['Date'].notnull()].groupby(['User_id', 'Merchant_id'], as_index=False).Date_received.count()
df_user_merchant_coupon_use.columns = ['User_id', 'Merchant_id', 'User_merchant_coupon_use']
df_user_merchant = pd.merge(df_user_merchant, df_user_merchant_coupon_use, on=['User_id', 'Merchant_id'], how='outer')
df_user_merchant.User_merchant_coupon_use.fillna(0, inplace=True)
df_user_merchant.User_merchant_coupon_use = df_user_merchant.User_merchant_coupon_use.astype(int)
df_user_merchant.head(2)

,User_id,Merchant_id,User_merchant_coupon_counts,User_merchant_coupon_nouse,User_merchant_coupon_use
0,4,1433,1,1,0
1,4,1469,1,1,0


#### 用户领取商家的优惠券后核销率 User_merchant_coupon_rate

In [37]:
df_user_merchant['User_merchant_coupon_rate'] = df_user_merchant.User_merchant_coupon_use / df_user_merchant.User_merchant_coupon_counts
df_user_merchant.head(2)

,User_id,Merchant_id,User_merchant_coupon_counts,User_merchant_coupon_nouse,User_merchant_coupon_use,User_merchant_coupon_rate
0,4,1433,1,1,0,0.0
1,4,1469,1,1,0,0.0


### 环境特征

#### 月份 Month

In [38]:
df_off.head(3)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,1,0.950000


In [39]:
def parse_month(item):
    time_str = str(item)
    month = time_str[4:6]
    return int(month)

In [40]:
df_off['Month'] = df_off.Date_received.apply(parse_month)
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate,Month
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667,5
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000,2


In [41]:
df_test['Month'] = df_test.Date_received.apply(parse_month)
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Coupon_type,Coupon_rate,Month
0,4129537,450,9983,30:5,1.0,20160712,1,0.833333,7
1,6949378,1300,3429,30:5,NaN,20160706,1,0.833333,7


#### 星期几 Weekday

In [42]:
def parse_weekday(item):
    time_str = str(item)
    year = time_str[:4]
    month = time_str[4:6]
    day = time_str[6:8]
    
    time_date = date(int(year), int(month), int(day))
    
    return time_date.weekday() + 1

In [43]:
df_off['Weekday'] = df_off.Date_received.apply(parse_weekday)
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate,Month,Weekday
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667,5,6
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000,2,3


In [44]:
df_test['Weekday'] = df_test.Date_received.apply(parse_weekday)
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Coupon_type,Coupon_rate,Month,Weekday
0,4129537,450,9983,30:5,1.0,20160712,1,0.833333,7,2
1,6949378,1300,3429,30:5,NaN,20160706,1,0.833333,7,3


#### 是否是周末 weekend

In [45]:
def parse_weekend(item):
    weekday = int(item)
    
    if weekday == 6 or weekday == 7:
        return 1
    else:
        return 0

In [46]:
df_off['Weekend'] = df_off.Weekday.apply(parse_weekend)
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate,Month,Weekday,Weekend
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667,5,6,1
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000,2,3,0


In [47]:
df_test['Weekend'] = df_test.Weekday.apply(parse_weekend)
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Coupon_type,Coupon_rate,Month,Weekday,Weekend
0,4129537,450,9983,30:5,1.0,20160712,1,0.833333,7,2,0
1,6949378,1300,3429,30:5,NaN,20160706,1,0.833333,7,3,0


### 创建标签

In [48]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate,Month,Weekday,Weekend
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667,5,6,1
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000,2,3,0


In [49]:
def parse_label(row):
    if str(row['Date']) != 'nan':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

In [50]:
df_off['Label'] = df_off.apply(parse_label, axis=1)
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Coupon_type,Coupon_rate,Month,Weekday,Weekend,Label
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,1,0.866667,5,6,1,0
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,1,0.950000,2,3,0,0


### 用户、商户、优惠劵特征汇总

In [51]:
df_train = df_off[['Label', 'Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Coupon_type', 'Coupon_rate', 'Distance', 'Month', 'Weekday', 'Weekend']].copy()
df_train.head(2)

,Label,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend
1,0,20160528.0,1439408,4663,11002.0,1,0.866667,1.0,5,6,1
2,0,20160217.0,1439408,2632,8591.0,1,0.950000,0.0,2,3,0


In [52]:
df_train.isnull().sum()

Label                 0
Date_received         0
User_id               0
Merchant_id           0
Coupon_id             0
Coupon_type           0
Coupon_rate           0
Distance         106003
Month                 0
Weekday               0
Weekend               0
dtype: int64

In [53]:
df_train.Distance.fillna(df_train.Distance.mean(), inplace=True)

In [54]:
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Coupon_type,Coupon_rate,Month,Weekday,Weekend
0,4129537,450,9983,30:5,1.0,20160712,1,0.833333,7,2,0
1,6949378,1300,3429,30:5,NaN,20160706,1,0.833333,7,3,0


In [55]:
df_test = df_test[['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Coupon_type', 'Coupon_rate', 'Distance', 'Month', 'Weekday', 'Weekend']].copy()
df_test.head(2)

,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend
0,20160712,4129537,450,9983,1,0.833333,1.0,7,2,0
1,20160706,6949378,1300,3429,1,0.833333,NaN,7,3,0


In [56]:
df_user.head(2)

,User_id,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate
0,4,2,2.0,0.0,0.0
1,35,4,4.0,0.0,0.0


In [57]:
df_train = pd.merge(df_train, df_user, on='User_id', how='left')
df_train.head(2)

,Label,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate
0,0,20160528.0,1439408,4663,11002.0,1,0.866667,1.0,5,6,1,5,4.0,1.0,0.2
1,0,20160217.0,1439408,2632,8591.0,1,0.950000,0.0,2,3,0,5,4.0,1.0,0.2


In [58]:
df_test = pd.merge(df_test, df_user, on='User_id', how='left')
df_test.head(2)

,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate
0,20160712,4129537,450,9983,1,0.833333,1.0,7,2,0,2.0,2.0,0.0,0.0
1,20160706,6949378,1300,3429,1,0.833333,NaN,7,3,0,1.0,0.0,1.0,1.0


In [59]:
df_test.isnull().sum()

Date_received              0
User_id                    0
Merchant_id                0
Coupon_id                  0
Coupon_type                0
Coupon_rate                0
Distance               12064
Month                      0
Weekday                    0
Weekend                    0
Coupon_counts           5590
Coupon_nouse_counts     5590
Coupon_use_counts       5590
Coupon_use_rate         5590
dtype: int64

In [60]:
df_test.Distance.fillna(df_test.Distance.mean(), inplace=True)
df_test.fillna(0, inplace=True)

In [61]:
df_coupon.head(2)

,Coupon_id,Coupon_history_counts,Coupon_hisotry_use_counts,Coupon_history_rate
0,1.0,5,1,0.2
1,2.0,2,1,0.5


In [62]:
df_train = pd.merge(df_train, df_coupon, on='Coupon_id', how='left')
df_train.head(2)

,Label,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate,Coupon_history_counts,Coupon_hisotry_use_counts,Coupon_history_rate
0,0,20160528.0,1439408,4663,11002.0,1,0.866667,1.0,5,6,1,5,4.0,1.0,0.2,7730,15,0.001940
1,0,20160217.0,1439408,2632,8591.0,1,0.950000,0.0,2,3,0,5,4.0,1.0,0.2,31,4,0.129032


In [63]:
df_test = pd.merge(df_test, df_coupon, on='Coupon_id', how='left')

In [64]:
df_test.shape

(113640, 17)

In [65]:
df_test.isnull().sum()

Date_received                    0
User_id                          0
Merchant_id                      0
Coupon_id                        0
Coupon_type                      0
Coupon_rate                      0
Distance                         0
Month                            0
Weekday                          0
Weekend                          0
Coupon_counts                    0
Coupon_nouse_counts              0
Coupon_use_counts                0
Coupon_use_rate                  0
Coupon_history_counts        95053
Coupon_hisotry_use_counts    95053
Coupon_history_rate          95053
dtype: int64

In [66]:
df_test.fillna(0, inplace=True)

In [67]:
df_merchant.head(2)

,Merchant_id,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate
0,2,7,7.0,0.0,0.0
1,3,10,9.0,1.0,0.1


In [68]:
df_train = pd.merge(df_train, df_merchant, on='Merchant_id', how='left')
df_train.head(2)

,Label,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,...,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate,Coupon_history_counts,Coupon_hisotry_use_counts,Coupon_history_rate,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate
0,0,20160528.0,1439408,4663,11002.0,1,0.866667,1.0,5,6,...,4.0,1.0,0.2,7730,15,0.001940,15191,15115.0,76.0,0.005003
1,0,20160217.0,1439408,2632,8591.0,1,0.950000,0.0,2,3,...,4.0,1.0,0.2,31,4,0.129032,43,39.0,4.0,0.093023


In [69]:
df_test = pd.merge(df_test, df_merchant, on='Merchant_id', how='left')
df_test.head(2)

,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend,...,Coupon_nouse_counts,Coupon_use_counts,Coupon_use_rate,Coupon_history_counts,Coupon_hisotry_use_counts,Coupon_history_rate,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate
0,20160712,4129537,450,9983,1,0.833333,1.00000,7,2,0,...,2.0,0.0,0.0,0.0,0.0,0.0,63042.0,61581.0,1461.0,0.023175
1,20160706,6949378,1300,3429,1,0.833333,2.32804,7,3,0,...,0.0,1.0,1.0,0.0,0.0,0.0,100.0,83.0,17.0,0.170000


In [70]:
df_test.isnull().sum()

Date_received                     0
User_id                           0
Merchant_id                       0
Coupon_id                         0
Coupon_type                       0
Coupon_rate                       0
Distance                          0
Month                             0
Weekday                           0
Weekend                           0
Coupon_counts                     0
Coupon_nouse_counts               0
Coupon_use_counts                 0
Coupon_use_rate                   0
Coupon_history_counts             0
Coupon_hisotry_use_counts         0
Coupon_history_rate               0
Merchant_coupon_counts          768
Merchant_coupon_nouse_counts    768
Merchant_coupon_use_counts      768
Merchant_coupon_use_rate        768
dtype: int64

In [71]:
df_test.fillna(0, inplace=True)

In [72]:
df_user_merchant.head(2)

,User_id,Merchant_id,User_merchant_coupon_counts,User_merchant_coupon_nouse,User_merchant_coupon_use,User_merchant_coupon_rate
0,4,1433,1,1,0,0.0
1,4,1469,1,1,0,0.0


In [73]:
df_train = pd.merge(df_train, df_user_merchant, on=['User_id', 'Merchant_id'], how='left')
df_train.head(2)

,Label,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,...,Coupon_hisotry_use_counts,Coupon_history_rate,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate,User_merchant_coupon_counts,User_merchant_coupon_nouse,User_merchant_coupon_use,User_merchant_coupon_rate
0,0,20160528.0,1439408,4663,11002.0,1,0.866667,1.0,5,6,...,15,0.001940,15191,15115.0,76.0,0.005003,1,1,0,0.00
1,0,20160217.0,1439408,2632,8591.0,1,0.950000,0.0,2,3,...,4,0.129032,43,39.0,4.0,0.093023,4,3,1,0.25


In [74]:
df_test = pd.merge(df_test, df_user_merchant, on=['User_id', 'Merchant_id'], how='left')
df_test.head(2)

,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend,...,Coupon_hisotry_use_counts,Coupon_history_rate,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate,User_merchant_coupon_counts,User_merchant_coupon_nouse,User_merchant_coupon_use,User_merchant_coupon_rate
0,20160712,4129537,450,9983,1,0.833333,1.00000,7,2,0,...,0.0,0.0,63042.0,61581.0,1461.0,0.023175,1.0,1.0,0.0,0.0
1,20160706,6949378,1300,3429,1,0.833333,2.32804,7,3,0,...,0.0,0.0,100.0,83.0,17.0,0.170000,NaN,NaN,NaN,NaN


In [75]:
df_test.shape

(113640, 25)

In [76]:
df_test.isnull().sum()

Date_received                       0
User_id                             0
Merchant_id                         0
Coupon_id                           0
Coupon_type                         0
Coupon_rate                         0
Distance                            0
Month                               0
Weekday                             0
Weekend                             0
Coupon_counts                       0
Coupon_nouse_counts                 0
Coupon_use_counts                   0
Coupon_use_rate                     0
Coupon_history_counts               0
Coupon_hisotry_use_counts           0
Coupon_history_rate                 0
Merchant_coupon_counts              0
Merchant_coupon_nouse_counts        0
Merchant_coupon_use_counts          0
Merchant_coupon_use_rate            0
User_merchant_coupon_counts     82155
User_merchant_coupon_nouse      82155
User_merchant_coupon_use        82155
User_merchant_coupon_rate       82155
dtype: int64

In [77]:
df_test.fillna(0, inplace=True)

In [78]:
df_train.shape, df_test.shape

((1053282, 26), (113640, 25))

In [79]:
df_train.Label.value_counts()

0    988887
1     64395
Name: Label, dtype: int64

### 划分数据集

In [100]:
train = df_train[df_train['Date_received'] < 20160516].copy()
valid = df_train[df_train['Date_received'] >= 20160516].copy()

train.shape, valid.shape

((800696, 26), (252586, 26))

### 模型选择

In [82]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics.scorer import roc_auc_scorer

In [83]:
def lr_model(X, Y):
    pipline = Pipeline([
        ('scaler', StandardScaler()),
        ('lr', SGDClassifier(loss='log', max_iter=10))
    ])
    
    search = {
        'lr__alpha':[0.01, 0.001, 0.0001],
    }
    
    grid_search = GridSearchCV(pipline, search, scoring=roc_auc_scorer, cv=3, verbose=2)
    
    grid_search.fit(X, Y)
    
    print('Best auc:', grid_search.best_score_)
        
    return grid_search.best_estimator_

In [88]:
best_model = lr_model(train.iloc[:, 2:], train.iloc[:, 0])

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] lr__alpha=0.01 ..................................................
[CV] ................................... lr__alpha=0.01, total=   2.3s
[CV] lr__alpha=0.01 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


[CV] ................................... lr__alpha=0.01, total=   2.2s
[CV] lr__alpha=0.01 ..................................................
[CV] ................................... lr__alpha=0.01, total=   2.2s
[CV] lr__alpha=0.001 .................................................
[CV] .................................. lr__alpha=0.001, total=   2.3s
[CV] lr__alpha=0.001 .................................................
[CV] .................................. lr__alpha=0.001, total=   2.3s
[CV] lr__alpha=0.001 .................................................
[CV] .................................. lr__alpha=0.001, total=   2.2s
[CV] lr__alpha=0.0001 ................................................
[CV] ................................. lr__alpha=0.0001, total=   2.3s
[CV] lr__alpha=0.0001 ................................................
[CV] ................................. lr__alpha=0.0001, total=   2.3s
[CV] lr__alpha=0.0001 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   24.0s finished


Best auc: 0.992454985014


In [107]:
proba_test = best_model.predict_proba(valid.iloc[:, 2:])[:, 1]

In [131]:
valid_auc = roc_auc_score(valid['Label'], proba_test)
print('valid_auc:', valid_auc)

valid_auc: 0.98451944062


In [109]:
valid['Proba'] = proba_test

In [136]:
sum = 0
num = 0

for coupon_id in valid.Coupon_id.unique():
    sub = valid[valid['Coupon_id'] == coupon_id]
    
    prob = sub['Proba']
    label = sub['Label']
    
    if len(sub.Label.unique()) != 2:
        continue
    
    auc = roc_auc_score(label, prob)
    sum += auc
    num += 1
    
mean_auc = sum / num

print('mean_auc: ', mean_auc)

mean_auc:  0.928794147174


### 输出结果

In [138]:
df_sample = pd.read_csv('data/sample_submission.csv')
df_sample.head(2)

,10000,20000,20160916,0.8
0,10001,20000,20160917,0.7
1,10000,20000,20160919,0.6


In [141]:
df_final = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')
df_final.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706


In [142]:
df_test.head(1)

,Date_received,User_id,Merchant_id,Coupon_id,Coupon_type,Coupon_rate,Distance,Month,Weekday,Weekend,...,Coupon_hisotry_use_counts,Coupon_history_rate,Merchant_coupon_counts,Merchant_coupon_nouse_counts,Merchant_coupon_use_counts,Merchant_coupon_use_rate,User_merchant_coupon_counts,User_merchant_coupon_nouse,User_merchant_coupon_use,User_merchant_coupon_rate
0,20160712,4129537,450,9983,1,0.833333,1.0,7,2,0,...,0.0,0.0,63042.0,61581.0,1461.0,0.023175,1.0,1.0,0.0,0.0


In [151]:
final_prob = best_model.predict_proba(df_test.iloc[:, 1:])[:, 1]

In [153]:
df_final['Probability'] = final_prob

In [154]:
df_final = df_final[['User_id', 'Coupon_id', 'Date_received', 'Probability']]
df_final.head(1)

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,0.009942


In [155]:
df_final.to_csv('zhangshulin.csv', header=False, index=False)